Importing Python packages

In [35]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from Utils.preprocessing import *

Importing datasets

In [29]:
path = './Datasets/Training/Large'
file_names = get_file_names(folder_path=path)
file_names[9]

File names:  ['Sample35.csv', 'Sample44.csv', 'Sample39.csv', 'Sample43.csv', '._.DS_Store', 'Sample36.csv', 'Sample38.csv', 'Sample33.csv', 'Sample42.csv', 'Sample40.csv', 'Sample37.csv', 'Sample34.csv', 'Sample41.csv']
Number of files:  13


'Sample40.csv'

In [30]:
df_path = path + '/' + file_names[0]
df = pd.read_csv(df_path)
df

,Time(s),Flow_Rate(ml/m),Upstream_Pressure(psi),Downstream_Pressure(psi)
0,0.0,-0.627256,0.25625,-0.70000
1,0.1,5.492969,0.73125,0.59375
2,0.2,4.999402,-0.71250,-1.35000
3,0.3,5.196829,0.80625,0.62500
4,0.4,2.136717,0.66875,0.58750
...,...,...,...,...
2395,239.5,94.236225,7.93750,0.56875
2396,239.6,88.116000,7.15625,-0.61875
2397,239.7,92.952952,6.33125,-1.21250
2398,239.8,98.480897,5.55625,-1.24375


Fixing NaN values in datatable rows

In [31]:
df = fix_invalid_rows(df)

Removing useless columns

In [26]:
#Pressure Drop (Upstream Pressure – Downstream Pressure)
#we identify the filter as clogged whenever the pressure drop is higher than 20 psi.

In [32]:
UP_Pessure = np.array(df['Upstream_Pressure(psi)'])
D_Pressure = np.array(df['Downstream_Pressure(psi)'])

drop_pressure = UP_Pessure-D_Pressure
drop_pressure

array([0.95625, 0.1375 , 0.6375 , ..., 7.54375, 6.8    , 6.5375 ])

In [33]:
df['pressure_drop'] = drop_pressure
px.scatter(data_frame= df, x = 'Time(s)', y = 'pressure_drop')

In [45]:
# training model with Logistic Regression method
model = LogisticRegression(random_state=0)

# removing unecessary columns for training and assigning results to a new table
X = df.drop(['Upstream_Pressure(psi)', 'Downstream_Pressure(psi)', 'Flow_Rate(ml/m)'], axis = 1)
y = np.zeros(len(X))
X['labels'] = y
X.loc[X['pressure_drop'] >= 20, 'labels'] = 1
y = X['labels']
X = X.drop(['labels'], axis = 1)

In [56]:
model.fit(X,y)
df_path = path + '/' + file_names[5]
df_path

test = pd.read_csv(df_path)

UP_Pessure = np.array(test['Upstream_Pressure(psi)'])
D_Pressure = np.array(test['Downstream_Pressure(psi)'])

drop_pressure = UP_Pessure-D_Pressure
test = test.drop(['Flow_Rate(ml/m)', 'Upstream_Pressure(psi)', 'Downstream_Pressure(psi)'], axis = 1)
test['pressure_drop'] = drop_pressure
test

labels = model.predict(test)
labels

test['labels'] = labels
ax = px.scatter(data_frame=test, x = 'Time(s)', y = 'pressure_drop', color='labels')
ax.show()

